<a href="https://colab.research.google.com/github/hollai465/VisualizationRedesign/blob/main/visualizationRedesign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
#pip installations
!pip install ipywidgets
!pip install plotly
!pip install dash
!pip install dash-table

In [120]:
#installations
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

import dash
from dash import dcc, html
import dash_table
from dash.dependencies import Input, Output
import logging





In [43]:
#upload data
df = pd.read_csv("betterdamdata.csv")
df.head()

,DAM_ID,Project name,Lon_Cleaned,LAT_cleaned,Country,people,lastOb,month,area
0,27,Keeyask,-95.179673,56.349493,Canada,4249.0,"Dec 1, 2024",jan,0.912
1,27,Keeyask,-95.179673,56.349493,Canada,4249.0,"Dec 1, 2024",feb,0.778
2,27,Keeyask,-95.179673,56.349493,Canada,4249.0,"Dec 1, 2024",apr,0.597
3,27,Keeyask,-95.179673,56.349493,Canada,4249.0,"Dec 1, 2024",mar,0.580
4,27,Keeyask,-95.179673,56.349493,Canada,4249.0,"Dec 1, 2024",may,0.558


In [170]:


average_people = df["people"].mean()

df_temp = df.copy()

df_temp["people"] = df_temp["people"].fillna(average_people)

app = dash.Dash(__name__)

# Create the Map Visualization
fig_map = px.scatter_mapbox(
    df_temp,
    lat="LAT_cleaned",
    lon="Lon_Cleaned",
    hover_name="Project name",
    hover_data={
        "DAM_ID": False,
        "Country": False,
        "people": False,
        "LAT_cleaned": False,
        "Lon_Cleaned": False,
    },
    size="people",
    size_max=10,
    color = "Country",
    color_discrete_map={
        "Canada": "blue",
        "Argentina": "red",
        "Brazil": "green"
    },
    zoom=2,
    height=600,

)

fig_map.update_layout(
    mapbox_style="open-street-map",
    margin={"r":0,"t":0,"l":0,"b":0}
)

default_table = {
    'Name': df['Project name'][0],
    'People': df['people'][0],
    'Last Observed': df['lastOb'][0],
    'Country': df['Country'][0]
}

app.layout = html.Div([
    # Title
    html.H1("Dam Visualization Dashboard", style={'textAlign': 'center'}),

    # Create a row for table + plot and map
    html.Div([
        # Column 1: Table on top and Line Plot below (50% width)
        html.Div([
            html.Div(id='dam-table', style={'margin-bottom': '0px','width': '100%','padding': '10px'}),  # Adding margin to reduce the gap
            dcc.Graph(id='line-plot', figure={},style={'width': '100%','padding': '10px'}),
        ], style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'top', 'padding': '10px','padding-top:':'0px'}),

        # Column 2: Map on top and Search below (50% width)
        html.Div([
            html.Div([
                dcc.Input(
                    id='search-bar',
                    type='text',
                    placeholder='Search for a dam...',
                    #debounce=500  # Wait for 500ms after the last keystroke
                ),
            ], style={'padding': '10px', 'margin-top': '0px','width': '100%'}),
            dcc.Graph(id='map', figure=fig_map,style={'width': '100%','padding': '10px'}),
        ], style={'padding': '10px','width': '50%', 'display': 'inline-block'}),
    ], style={'display': 'flex', 'justify-content': 'space-between', 'align-items': 'flex-start','backgroundColor': '#003F88'}),
    dcc.Store(id='debug-store')
], style = {'backgroundColor': '#00296B'})




@app.callback(
    [Output('line-plot', 'figure'),
     Output('dam-table', 'children'),
    Output('debug-store', 'data')],
    [Input('map', 'clickData')],
    [Input('search-bar', 'value')]
)
def update_visuals(clickData,search_value):
    if clickData is None and search_value is None:
      project_name = "Keeyask"
    else:
      if search_value:
        dam_data = df_temp[df_temp["Project name"].str.contains(search_value, case=False, na=False)]
        if not dam_data.empty:
          project_name = dam_data['Project name'].iloc[0]
        else:
          project_name = "Keeyask"
      else:
          project_name = clickData['points'][0]['hovertext']

    dam_data = df_temp[df_temp["Project name"] == project_name]

    dam_data = dam_data.sort_values(by="month")

      # Prepare surface area data for the line plot
    surface_area = dam_data["area"].values
    months = dam_data["month"].values

      # Create the line plot
    line_fig = go.Figure()
    line_fig.add_trace(go.Scatter(
        x=months,
        y=surface_area,
        mode='lines+markers',
        name="Surface Area",
    ))
    line_fig.update_layout(
        title="Surface Area Over Time",
        xaxis_title="Month",
        yaxis_title="Surface Area",
        showlegend=False
    )

    # Create the table data for the selected dam as key-value pairs
    dam_info = dam_data[['Project name', 'Country', 'people','lastOb']].iloc[0].to_dict()

    # Prepare the data for the DataTable
    table_data = [{
        'Attribute': key,
        'Value': value
    } for key, value in dam_info.items()]

      # Create the DataTable layout
    table_content = dash_table.DataTable(
        columns=[
            {"name": "Attribute", "id": "Attribute"},
            {"name": "Value", "id": "Value"}
        ],
        data=table_data,
        style_table={
            'overflowY': 'auto',
            },
        style_cell={
            'textAlign': 'left',
            'padding': '10px'
        },
        style_header={
            'display': 'none',
        }
    )
    debug_data = {'project_name': project_name, 'dam_info': dam_info}
    return line_fig, table_content, debug_data

if __name__ == '__main__':
    app.run(debug=True)



DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:8050
DEBUG:urllib3.connectionpool:http://127.0.0.1:8050 "GET /_alive_5fe783a4-45c7-4dad-b996-810aacbf2bb6 HTTP/1.1" 200 5


<IPython.core.display.Javascript object>